In [97]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import glob

# 画像の保存
import os
import pprint
import time
import urllib.error
import urllib.request

# Jupyter上にHTMLを表示する
from IPython.display import HTML

In [98]:
def output_html(videos, top_n=10, shuffle=True):
    html = '<h1>動画一覧を表示</h1>'
    html += '<div style="float:left;">'
    if shuffle:
        indexes = np.random.choice(videos.shape[0], size=top_n, replace=False)
    else:
        indexes = np.arange(videos.shape[0])[0:top_n]        
    
    for i in indexes:            
        html += ('<img src="'+np.array(videos['Thumbnail'])[i] +' "alt="取得できませんでした" width="100">')
        # サンプルIDを入れておく
        html += ('<a href="https://www.youtube.com/watch?v='+np.array(videos['Id'])[i]+'">#'+str(i+1)+'. '+np.array(videos['Title'])[i]+'</a><br>')
    html += '</div>'
    return html

In [99]:
# 取得した全動画情報を抽出する

holo_filenames = glob.glob("output/hololive/*")
# holo_filenames = glob.glob("output/okakoro/*")

for i, holo_filename in enumerate(holo_filenames):
    holo_videos = pd.read_csv(holo_filename, index_col=0)
    # テーブルを結合する
    if i == 0:
        all_holo_videos = holo_videos
    else:
        all_holo_videos = pd.concat([all_holo_videos, holo_videos])

# DescriptionがNanのケースがあるため補完する
all_holo_videos = all_holo_videos.fillna('')
# indexを振り直す
all_holo_videos = all_holo_videos.reset_index()

print("全動画数:", all_holo_videos.shape[0])
# HTML(output_html(all_holo_videos))

全動画数: 7165


In [100]:
# フィルタリング1：再生時間によるフィルタリング

# 再生時間で曲っぽく無いものは除外
# 1分〜8分に設定（1コーラスを考慮し）
filetered_time_videos = all_holo_videos[np.logical_and(all_holo_videos['Duration']<480, 
                                                            all_holo_videos['Duration']>60)]

print("再生時間によるフィルタリング後：", filetered_time_videos.shape[0])
# HTML(output_html(filetered_time_videos))

再生時間によるフィルタリング後： 517


In [101]:
# フィルタリング2：カテゴリによるフィルタリング

# 音楽はCategoryId=10(音楽), 24(エンターテイメント)以外のものは無いと仮定する
filetered_category_videos = filetered_time_videos[np.logical_or(filetered_time_videos['CategoryId']==10,
                                                                filetered_time_videos['CategoryId']==24)]

print("カテゴリによるフィルタリング後：", filetered_category_videos.shape[0])
# HTML(output_html(filetered_time_category))

カテゴリによるフィルタリング後： 421


In [102]:
# フィルタリング3：Descriptionによるフィルタリング
filtered_desc_index_plus = np.zeros(n)
filtered_desc_index_minus = np.zeros(n)
descs = np.array(filetered_category_videos['Description'])

# Descriptionに"歌ってみた"などが入る動画は高い可能性で歌動画と判断
desc_keywords_jap = ['歌', '作詞', '作曲', '編曲', '原曲', '本家', '歌唱', 'ボーカル', 'ミックス']
desc_keywords_eng = ['SONG', 'MV', 'VOCAL', 'MIX']
desc_keywords = []
desc_keywords.extend(desc_keywords_jap)
desc_keywords.extend(desc_keywords_eng)
for i, desc in enumerate(descs):
    desc_upper = desc.upper()
    for keyword in desc_keywords:
        if keyword in desc:
            filtered_desc_index_plus[i] += 1

# 逆に"実況"や"アニメ"など入っている場合は歌動画ではないと判断
# しかし、概要欄の紹介などで入る可能性も高いため一旦保留とする
# desc_stopwords = []
# for i, desc in enumerate(descs):
#     desc_upper = desc.upper()
#     for keyword in desc_stopwords:
#         if keyword in desc:
#             filtered_desc_index_minus[i] += 1

filetered_desc_videos = filetered_category_videos[filtered_desc_index_plus > 0]
print("概要欄によるフィルタリング後：", filetered_desc_videos.shape[0])
# HTML(output_html(filetered_desc_videos))

概要欄によるフィルタリング後： 273


In [103]:
# ランキングにして可視化

sorted_videos = filetered_desc_videos.sort_values(by='Viewcount', ascending=False)
HTML(output_html(sorted_videos, shuffle=False))